In [ ]:
import pandas as pd 
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img 
# from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
import random
import os 

import tensorflow as tf
import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

import cv2
import random
from os import listdir
from numpy import save, load
import time

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import copy
import matplotlib.cm as cm
import scipy.stats as stats
from sklearn.metrics import roc_curve, auc

In [33]:
def display_heatmap(img_, heatmap, alpha=0.4):  
    #Convert a grayscale image to an rgb image
    img_ = img_
    img = np.zeros((img_.shape[0],img_.shape[1],3))
    img[:,:,0] = img_[:,:,0]
    img[:,:,1] = img_[:,:,0]
    img[:,:,2] = img_[:,:,0]

    # Adjust by pixel value from 0 to 255
    img = np.uint8(255 * img)
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize attention map
    jet = cm.get_cmap("jet")
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = tf.keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = tf.keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    return superimposed_img

In [ ]:
last_conv_layer_num = -6 # depends on model structure
image_height = 256 # input image shape
image_width = 256

In [104]:
def grad_cam(input_model, image):
    preds = input_model.predict(image)
    predicted_label = np.argmax(preds[0])
    heatmap_model = tf.keras.models.Model([input_model.inputs], [input_model.layers[last_conv_layer_num].output, input_model.output])        
    
    with tf.GradientTape() as tape:
        last_conv_outputs, predictions = heatmap_model(image) 
        output = predictions[:, predicted_label]  
        
    grads = tape.gradient(output, last_conv_outputs)      
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))  
    
    last_conv_outputs = last_conv_outputs.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i, w in enumerate(pooled_grads):                    
        last_conv_outputs[:,:,i] *= w
        
    heatmap = np.mean(last_conv_outputs, axis=-1) 
    
    heatmap = np.maximum(heatmap, 0) 
    maxval = np.max(heatmap)
    if np.max(heatmap)==0:
        maxval=1e-5   
    heatmap /= maxval
    heatmap = cv2.resize(heatmap, (image_height,image_width))
    
    return heatmap

In [36]:
def grad_ram(input_model, image): # regression activation map
    preds = input_model.predict(image)
    predicted_label = np.argmax(preds[0])
    heatmap_model = tf.keras.models.Model([input_model.inputs], [input_model.layers[last_conv_layer_num].output, input_model.output])        
    
    with tf.GradientTape() as tape:
        last_conv_outputs, predictions = heatmap_model(image) 
        output = predictions[:, predicted_label]  
        
    grads = tape.gradient(output, last_conv_outputs)      
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))  
    
    last_conv_outputs = last_conv_outputs.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i, w in enumerate(pooled_grads):                    
        last_conv_outputs[:,:,i] *= w
        
    heatmap = np.mean(last_conv_outputs, axis=-1) 
    
    heatmap = np.abs(heatmap)             
    heatmap /= np.max(heatmap)
    heatmap = cv2.resize(heatmap, (image_height,image_width))
    
    return heatmap

In [37]:
def grad_ramp(input_model, image): # regression activation map by pixel
    preds = input_model.predict(image)
    predicted_label = np.argmax(preds[0])
    heatmap_model = tf.keras.models.Model([input_model.inputs], [input_model.layers[last_conv_layer_num].output, input_model.output])        
    
    with tf.GradientTape() as tape:
        last_conv_outputs, predictions = heatmap_model(image) 
        output = predictions[:, predicted_label]  
        
    grads = tape.gradient(output, last_conv_outputs)      
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))  
    
    grads0 = grads[0]
    
    last_conv_outputs = last_conv_outputs.numpy()[0]
    
    pooled_grads = pooled_grads.numpy()
    for k in range(grads0.shape[2]):
        last_conv_outputs[:,:,k] = np.multiply(last_conv_outputs[:,:,k],grads0[:,:,k])        
        
    heatmap = np.mean(last_conv_outputs, axis=-1) 
    
    heatmap = np.abs(heatmap)
    heatmap /= np.max(heatmap)
    heatmap = cv2.resize(heatmap, (image_height,image_width))
    
    return heatmap